# Extracting Argument Premises Using One-Shot Prompting

This notebook applies one-shot prompting to extract argument premises from the data entries.
There are two prompts: one-shot and one-shot chain-of-thought.
There are two types of labels: individual annotator labels and majority vote labels.

In [ ]:
import os
import json
from tqdm import tqdm
from OllamaCached import llm_chat

In [ ]:
# setting environment variables for OllamaCached model
os.environ["OLLAMA_MODELS"] = "./models"
os.environ["OLLAMA_KEEP_ALIVE"] = "1m"

In [ ]:
# list of policy options associated with the data entries
policy_option_list = ['The municipality takes the lead and unburdens you',
                      'Inhabitants do it themselves',
                      'The market determines what is coming',
                      'Large-scale energy generation will occur in a small number of places',
                      'Betting on storage (Sudwest-Fryslan becomes the battery of the Netherlands)',
                      'Become a major energy supplier in the Netherlands']

In [ ]:
# extract the required data per annotator
def extract_data_per_annotator(annotator_number):
    file_path = f'data_extraction/personal_annotation_{annotator_number}.json'

    with open(file_path, 'r') as json_file:
        data = json.load(json_file)

    return data

In [ ]:
def apply_one_shot_per_annotator(annotator_number, data, training_example):
    """
    Applies one-shot prompting per individual annotator.
    Args:
        annotator_number: number of annotator
        data: required data
        training_example: training example used

    Returns: saves the results to location 'llm_responses/one_shot_majority_vote.json'

    """
    # path for saving the result
    file_path = f'llm_responses/one_shot_majority_vote.json'
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    number_of_trials = 10 # adjust as needed
    trial_collection = []

    for i in range (0, number_of_trials):
        print(f"Run number {i + 1}")
        output_collection = []
        for i, argument in enumerate(tqdm(data.get('english_text'), desc='Processing arguments', unit = 'argument')):

            # one-shot prompt
            one_shot_prompt = [
            {'role': 'system', 'content': f'''You are an expert in identifying arguments and their premises in a text. An argument is a set of claims in which one or more of them—the premises—are put forward so as to offer reasons for another claim, the conclusion. A text that does not provide support or reasons for a conclusion is not an argument. {training_example}'''},
            {'role': 'user', 'content': f'''Can you extract the words in this sentence that state the premise or premises of this text: {argument} or extract "None" if the text is not an argument.

            Return the following JSON with the premises found in the text:
            {{
                "premise_1": "",
                ...
            }}.'''}
        ]

            # one-shot chain-of-thought prompt
            one_shot_cot_prompt = [
            {'role': 'system', 'content': f'''You are an expert in identifying arguments and their premises in a text. An argument is a set of claims in which one or more of them—the premises—are put forward so as to offer reasons for another claim, the conclusion. A text that does not provide support or reasons for a conclusion is not an argument.'''},
            {'role': 'user', 'content': f'''Here is an example to illustrate the task: {training_example}
             Now, let's move to the actual task.'''},
            {'role': 'user', 'content': f'''Let's think step by step. You know that the text is related to this conclusion: {policy_option_list[data.get('question_id')[i] - 1]}. Can you extract the words in this sentence that state the premise or premises of this text: {argument} or extract "None" if the text is not an argument.
             Step 1, determine if the text provides any support or reasons for the conclusion. If it does not provide support or reasons for the conclusion, return {{"premise_1": "None"}} and don't execute the next steps.
             Step 2, If it does provide support or reasons, return the following JSON with the premises found in the text:
            {{
                "premise_1": "",
                ...
            }}.
             '''}]

            # adjust prompt message as needed
            output = llm_chat(model_name='llama2:latest',
                               message=one_shot_cot_prompt)

            output_collection.append(output)
            print(output)
        trial_collection.append(output_collection)

    # save results
    with open(file_path, 'w') as json_file:
        json.dump(trial_collection, json_file)

In [ ]:
# training example for one-shot method
# it is personalized to the annotated data of this research
# if data is annotated differently, this method may be subject to change
def create_training_example(annotator_number):
    if annotator_number == 2:
        return f"An example for this task is for input : {data.get('english_text')[0]}, the output is: {{'premise_1': {data.get('labels')[0][0]}}}."

    return f"An example for this task is for input : {data.get('english_text')[0]}, the output is: {{'premise_1': {data.get('labels')[0][0]}, 'premise_2': {data.get('labels')[0][1]}}}."

In [ ]:
# training example for one-shot chain-of-thought method
# it is personalized to the annotated data of this research
# if data is annotated differently, this method may be subject to change
def create_cot_training_example(annotator_number):
    if annotator_number == 2:
        return f"An example for this is for input: {data.get('english_text')[0]}:" \
               f"Step 1: the text does provide support or reasons for the conclusion. Therefore, execute the next step." \
               f"Step 2: return output: {{'premise_1': {data.get('labels')[0][0]}}}." \
               f"Now apply similar logic to other examples."

    return f"An example for this is for input: {data.get('english_text')[0]}:" \
               f"Step 1: the text does provide support or reasons for the conclusion. Therefore, execute the next step." \
               f"Step 2: return output: {{'premise_1': {data.get('labels')[0][0]}, 'premise_2': {data.get('labels')[0][1]}}}"

In [ ]:
# training example for one-shot chain-of-thoughts majority vote labels method
# it is personalized to the annotated data of this research
# if data is annotated differently, this method may be subject to change
%run aggregate_labels.ipynb
def create_majority_vote_training_example():
    # get majority vote labels
    aggregated_data = get_majority_vote_labels_dataset()

    return f"An example for this is for input: {data.get('english_text')[0]}:" \
               f"Step 1: the text does provide support or reasons for the conclusion. Therefore, execute the next step." \
               f"Step 2: return output: {{'premise_1': {aggregated_data[0][0]}, 'premise_2': {aggregated_data[0][1]}}}"


In [ ]:
# compute one-shot results for each of annotator
for i in range(0, 5):
    data = extract_data_per_annotator(i + 1)
    argument_data = list(data.get("english_text"))
    labels = list(data.get("labels"))

    training_example = create_training_example(i + 1)
    cot_training_example = create_cot_training_example(i + 1)
    majority_vote_training_example = create_majority_vote_training_example()
    # change training example used as needed
    apply_one_shot_per_annotator(i + 1, data, create_majority_vote_training_example())